The following notebook will explore any trends between the air quality data and the weather data. Both datasets contain hourly measurements.

In [38]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')


In [45]:
# Switch to data cleaning branch and read in cleaned air quality and weather dataframes
#!git   checkout 1-Data_cleaning
weather_data = pd.read_csv(r"C:\Users\user\PycharmProjects\CIV1498Project2\1-Data_cleaning\weather_data_cleaned.csv")
aq_data = pd.read_csv(r"")


,Datetime,TEMP_MEAN_(C),WDIR_VECT_(DEG),WDIR_UVEC_(DEG),WSPD_SCLR_(M/S),HUMIDITY_(%)
0,2019-01-01 01:00:00,0.8,272.7,273.2,10.0,67.1
1,2019-01-01 02:00:00,0.8,274.6,275.0,8.8,66.8
2,2019-01-01 03:00:00,1.2,274.9,274.8,8.4,66.0
3,2019-01-01 04:00:00,0.7,268.3,268.6,7.7,68.6
4,2019-01-01 05:00:00,1.1,263.5,263.6,10.4,69.6
